In [1]:
import json
import torch
import random
import numpy as np
import pandas as pd

In [2]:
def load_json(filepath = None):
    
    assert(filepath != None)
    
    f = open(filepath)
    data = json.load(f)
    f.close()
    
    return data

In [79]:
def get_da_type(item):
    gold = item['qa']['gold_inds']
    
    is_table = False
    is_text = False
    type_ = -1
    
    for key in gold:
        if 'table' in key:
            is_table = True
        if 'text' in key:
            is_text = True
    
    if is_table and is_text:
        type_ = 2
    elif is_table and not is_text:
        type_ = 0
    elif is_text and not is_table:
        type_ = 1
        
    return type_

In [105]:
def get_step(item):
    program = item['qa']['program']
    count = program.count('#')
    return count

In [120]:
def get_const(item):
    program_re = item['qa']['program_re']
    if 'const' in program_re:
        is_const = True
        print(program_re)
        print()
    else:
        is_const = False
    return is_const

In [103]:
item['qa']

{'question': 'what was the sum of the european papers net sales from 2006 to 2004',
 'answer': '4.4',
 'explanation': '',
 'ann_table_rows': [],
 'ann_text_rows': [40],
 'steps': [{'op': 'add2-1', 'arg1': '1.5', 'arg2': '1.4', 'res': '2.9'},
  {'op': 'add2-2', 'arg1': '#0', 'arg2': '1.5', 'res': '4.4'}],
 'program': 'add(1.5, 1.4), add(#0, 1.5)',
 'gold_inds': {'text_40': 'european papers net sales in 2006 were $ 1.5 bil- lion , compared with $ 1.4 billion in 2005 and $ 1.5 bil- lion in 2004 .'},
 'exe_ans': 4.4,
 'tfidftopn': {'text_0': 'printing papers net sales for 2006 decreased 3% ( 3 % ) from both 2005 and 2004 due principally to the sale of the u.s .',
  'text_20': 'coated papers net sales were $ 920 million in 2006 , $ 1.6 billion in 2005 and $ 1.4 billion in 2004 .'},
 'model_input': [['text_0',
   'printing papers net sales for 2006 decreased 3% ( 3 % ) from both 2005 and 2004 due principally to the sale of the u.s .'],
  ['text_20',
   'coated papers net sales were $ 920 mil

## NAED

In [13]:
NAED_full = load_json("./results/NAED/full_results.json")
NAED_wrong = load_json("./results/NAED/full_results_error.json")
NAED_nbest = load_json("./results/NAED/nbest_predictions.json")
NAED_predictions = load_json("./results/NAED/predictions.json")

In [129]:
for item in NAED_full:
    print(item['qa']['program'])
    print(item['qa']['predicted'])
    print('='*40)

subtract(5829, 5735)
['subtract(', '5829', '5735', ')', 'EOF']
divide(8.1, 56.0)
['divide(', '8.1', '56.0', ')', 'EOF']
subtract(153.7, 139.9), divide(#0, 139.9)
['subtract(', '153.7', '139.9', ')', 'divide(', '#0', '139.9', ')', 'EOF']
divide(121.4, 4187.8)
['divide(', '121.4', '4187.8', ')', 'EOF']
subtract(318.46, const_100), divide(#0, const_100), subtract(206.49, const_100), divide(#2, const_100), subtract(#1, #3)
['subtract(', '318.46', 'const_100', ')', 'divide(', '#0', 'const_100', ')', 'subtract(', '206.49', 'const_100', ')', 'divide(', '#2', 'const_100', ')', 'subtract(', '#1', '#3', ')', 'EOF']
subtract(92710000, 86842000), divide(#0, 86842000)
['subtract(', '100690000', '92710000', ')', 'divide(', '#0', '92710000', ')', 'EOF']
divide(463, 4612)
['divide(', '463', '4612', ')', 'EOF']
divide(301, 2575)
['divide(', '301', '2575', ')', 'EOF']
greater(286.61, 198.09)
['greater(', '286.61', '198.09', ')', 'EOF']
subtract(1136, 1171)
['subtract(', '1136', '1171', ')', 'EOF']
subtr

add(587, 580)
['add(', '587', '580', ')', 'EOF']
divide(89, 99), subtract(const_1, #0)
['subtract(', '110', '89', ')', 'divide(', '#0', '110', ')', 'EOF']
divide(0.4, 0.8)
['divide(', '0.4', '0.8', ')', 'EOF']
subtract(500.0, 240.9), divide(#0, 240.9)
['subtract(', '240.9', '2.6', ')', 'divide(', '#0', '10.3', ')', 'EOF']
subtract(29.7, 21.0), divide(#0, 21.0)
['subtract(', '29.7', '21.0', ')', 'divide(', '#0', '21.0', ')', 'EOF']
multiply(2229188, const_100), divide(#0, 2.50)
['multiply(', '2229188', '2.50%', ')', 'EOF']
subtract(313.45, const_100), divide(#0, const_100), subtract(198.18, const_100), divide(#2, const_100), subtract(#1, #3)
['subtract(', '313.45', 'const_100', ')', 'divide(', '#0', 'const_100', ')', 'subtract(', '198.18', 'const_100', ')', 'divide(', '#2', 'const_100', ')', 'subtract(', '#1', '#3', ')', 'EOF']
divide(772, 3693)
['divide(', '772', '3693', ')', 'EOF']
divide(47162, 83659)
['divide(', '47162', '83659', ')', 'EOF']
divide(684, 7001)
['divide(', '684', '700

In [37]:
NAED_wrong_idx_list = []
for item in NAED_wrong:
    NAED_wrong_idx_list.append(item['id'])

In [38]:
len(NAED_wrong_idx_list)

365

In [109]:
NAED_table_only_correct = []  # 0 
NAED_table_only_wrong = []    
NAED_text_only_correct = []   # 1
NAED_text_only_wrong = []
NAED_table_text_correct = []  # 2
NAED_table_text_wrong = []

NAED_1step_correct = []
NAED_1step_wrong = []
NAED_2step_correct = []
NAED_2step_wrong = []
NAED_nstep_correct = []
NAED_nstep_wrong = []

NAED_const_correct = []
NAED_const_wrong = []
NAED_no_const_correct = []
NAED_no_const_wrong = []

In [121]:
for item in NAED_full:
    idx = item['id']
    
    # whether correct or wrong
    if idx in NAED_wrong_idx_list:
        wrong_or_correct = 0
    else:
        wrong_or_correct = 1

    type_idx = get_da_type(item)
    
    if wrong_or_correct == 1:
        if type_idx == 0:
            NAED_table_only_correct.append(idx)
        elif type_idx == 1:
            NAED_text_only_correct.append(idx)
        else:
            NAED_table_text_correct.append(idx)
    else:
        if type_idx == 0:
            NAED_table_only_wrong.append(idx)
        elif type_idx == 1:
            NAED_text_only_wrong.append(idx)
        else:
            NAED_table_text_wrong.append(idx)
        
    count_step = get_step(item)

    if wrong_or_correct == 1:
        if count_step == 0:
            NAED_1step_correct.append(idx)
        elif count_step == 1:
            NAED_2step_correct.append(idx)
        else:
            NAED_nstep_correct.append(idx)
    else:
        if count_step == 0:
            NAED_1step_wrong.append(idx)
        elif count_step == 1:
            NAED_2step_wrong.append(idx)
        else:
            NAED_nstep_wrong.append(idx)
            
    is_const = get_const(item)
    if wrong_or_correct == 1:
        if is_const:
            NAED_const_correct.append(idx)
        else:
            NAED_no_const_correct.append(idx)
    else:
        if is_const:
            NAED_const_wrong.append(idx)
        else:
            NAED_no_const_wrong.append(idx)

subtract(divide(subtract(318.46, const_100), const_100), divide(subtract(206.49, const_100), const_100))

divide(455, const_7)

divide(add(add(703.1, 705.4), const_2), const_2)

divide(15.3, divide(139549, const_1000))

subtract(divide(subtract(311.81, const_100), const_100), divide(subtract(198.18, const_100), const_100))

add(add(4.6, 5.5), const_2)

subtract(divide(subtract(248.36, const_100), const_100), divide(subtract(236.20, const_100), const_100))

divide(add(950.4, add(957.4, 769.1)), const_3)

divide(multiply(766801, 40.85), const_1000000)

multiply(divide(10.4, 61.4), const_100)

divide(const_3, 401362)

multiply(multiply(690, const_1000000), 8.75%)

divide(multiply(2, const_1000000), 2779)

divide(subtract(137.90, const_100), const_100)

divide(subtract(121.48, const_100), const_100)

divide(subtract(224.65, const_100), const_100)

divide(945.5, multiply(1.4, const_1000))

subtract(exp(divide(subtract(208.14, 100), 100), divide(const_1, const_5)), const_1)

divide(subtract(

## Baseline

In [76]:
baseline_full = load_json("./results/baseline/full_results.json")
baseline_wrong = load_json("./results/baseline/full_results_error.json")
baseline_nbest = load_json("./results/baseline/nbest_predictions.json")
baseline_predictions = load_json("./results/baseline/predictions.json")

In [77]:
baseline_wrong_idx_list = []
for item in baseline_wrong:
    baseline_wrong_idx_list.append(item['id'])

In [78]:
len(baseline_wrong_idx_list)

379

In [113]:
baseline_table_only_correct = []  # 0 
baseline_table_only_wrong = []    
baseline_text_only_correct = []   # 1
baseline_text_only_wrong = []
baseline_table_text_correct = []  # 2
baseline_table_text_wrong = []

baseline_1step_correct = []
baseline_1step_wrong = []
baseline_2step_correct = []
baseline_2step_wrong = []
baseline_nstep_correct = []
baseline_nstep_wrong = []

baseline_const_correct = []
baseline_const_wrong = []
baseline_no_const_correct = []
baseline_no_const_wrong = []

In [114]:
for item in baseline_full:
    idx = item['id']
    
    # whether correct or wrong
    if idx in baseline_wrong_idx_list:
        wrong_or_correct = 0
    else:
        wrong_or_correct = 1

    type_idx = get_da_type(item)
    
    if wrong_or_correct == 1:
        if type_idx == 0:
            baseline_table_only_correct.append(idx)
        elif type_idx == 1:
            baseline_text_only_correct.append(idx)
        else:
            baseline_table_text_correct.append(idx)
    else:
        if type_idx == 0:
            baseline_table_only_wrong.append(idx)
        elif type_idx == 1:
            baseline_text_only_wrong.append(idx)
        else:
            baseline_table_text_wrong.append(idx)
        
    count_step = get_step(item)
    if wrong_or_correct == 1:
        if count_step == 0:
            baseline_1step_correct.append(idx)
        elif count_step == 1:
            baseline_2step_correct.append(idx)
        else:
            baseline_nstep_correct.append(idx)
    else:
        if count_step == 0:
            baseline_1step_wrong.append(idx)
        elif count_step == 1:
            baseline_2step_wrong.append(idx)
        else:
            baseline_nstep_wrong.append(idx)
            
    is_const = get_const(item)
    if wrong_or_correct == 1:
        if is_const:
            baseline_const_correct.append(idx)
        else:
            baseline_no_const_correct.append(idx)
    else:
        if is_const:
            baseline_const_wrong.append(idx)
        else:
            baseline_no_const_wrong.append(idx)

## Analysis

### data type

1. TEXT ONLY:
    * CORRECT:
        * NAED: 169    # higher
        * baseline: 163
    * WRONG:
        * NAED: 114
        * baseline: 120  # higher
        
2. TABLE ONLY:
    * CORRECT:
        * NAED: 533  # higher
        * baseline: 528
    * WRONG:
        * NAED: 173
        * baseline: 178  # higher
        
3. TEXT-TABLE:
    * CORRECT:
        * NAED: 80
        * baseline: 77
    * WRONG:
        * NAED: 78
        * baseline: 81

### Step

1. 1 step:
    * CORRECT:
        * NAED: 502
        * baseline: 485
    * WRONG:
        * NAED: 155
        * baseline: 172

2. 2 steps:
    * CORRECT:
        * NAED: 260
        * baseline: 259
    * WRONG:
        * NAED: 146
        * baseline: 147

3. more steps:
    * CORRECT:
        * NAED: 20
        * baseline: 24
    * WRONG:
        * NAED: 64
        * baseline: 60


In [ ]:
NAED_table_only_correct = []  # 0 
NAED_table_only_wrong = []    
NAED_text_only_correct = []   # 1
NAED_text_only_wrong = []
NAED_table_text_correct = []  # 2
NAED_table_text_wrong = []

print(len(NAED_1step_correct))
print(len(NAED_1step_wrong))
print(len(NAED_2step_correct))
print(len(NAED_2step_wrong))
print(len(NAED_nstep_correct))
print(len(NAED_nstep_wrong))

NAED_const_correct
NAED_const_wrong
NAED_no_const_correct
NAED_no_const_wrong

In [96]:
len(baseline_table_text_correct)

77

In [115]:
print(len(baseline_1step_correct))
print(len(baseline_1step_wrong))
print(len(baseline_2step_correct))
print(len(baseline_2step_wrong))
print(len(baseline_nstep_correct))
print(len(baseline_nstep_wrong))

485
172
259
147
24
60


In [116]:
print(len(NAED_const_correct))
print(len(NAED_const_wrong))
print(len(NAED_no_const_correct))
print(len(NAED_no_const_wrong))

58
110
724
255


In [130]:
print(len(baseline_const_correct))
print(len(baseline_const_wrong))
print(len(baseline_no_const_correct))
print(len(baseline_no_const_wrong))

62
106
706
273
